In [1]:
import pandas as pd
import numpy as np
import requests
import os
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Gather

In [2]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
folder_name = 'image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url_list = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv']

for url in url_list:
    response = requests.get(url)
    with open(os.path.join(folder_name , url.split('/')[-1]) , mode = 'wb') as file:
             file.write(response.content)

In [4]:
image_prediction = pd.read_csv('image_predictions/image-predictions.tsv' , sep = '\t')

### Additional Data via the Twitter API

In [5]:
# I used configparser for my API keys and secret.
# This code returns error if you do not input your API and secret info.

import configparser

config = configparser.ConfigParser()

config['consumer'] = {
    'key': 'KTXWZ3Ucy7o9xodDyHNM9expc',
    'secret': 'kspq1bKtllzCVCK8XIlEuINz7qAzOw9xufM0OT2PWGvxGwbJUw'
}
config['access'] = {
    'token': '1160345017220812800-vitA9IyHFSrHg8zJnK2MjwZWk4lAEI',
    'secret': '5CFzbBJw8NzpDOxOiGs79PnBURGGEqZWKqOb6dZuJZsnt',
}
with open('config.ini', 'w') as file:
    config.write(file)

In [6]:
# read my APi info from config.ini

consumer_key = config['consumer']['key']
consumer_secret = config['consumer']['secret']
access_token = config['access']['token']
access_secret = config['access']['secret']


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth , wait_on_rate_limit=True)

In [7]:
tweet_ids = twitter_archive.tweet_id.values

# Use timer to calculate how long this code takes to run
start = timer()

# Create a text file called tweet_json.txt
with open ('tweet_json.txt' , 'w') as outfile:
    for tweet_id in tweet_ids:
        try:
            # Get tweet informations
            tweet_info = api.get_status(tweet_id, tweet_mode='extended')
            #Write in the text file
            json.dump(tweet_info._json , outfile)
            # Put a indent for better readiness
            outfile.write('\n')
        
        # Error tweets
        except tweepy.TweepError as e:
            # Print tweet ID and error code description
            print ("Tweet ID:{}{}".format(str(tweet_id),e.reason))

# Timer calculation            
end = timer()
print(end - start , "seconds")

In [8]:
tweepy_data = pd.read_json('tweet_json.txt' , lines = True)

# Assess

## twitter_archive

In [9]:
twitter_archive.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [10]:
# tweet id has wrong datatype
# timestamp has wrong datatype

twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [11]:
# some strange names
twitter_archive.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Oliver        11
            ... 
Eevee          1
Pete           1
Eugene         1
Siba           1
Sojourner      1
Name: name, Length: 957, dtype: int64

In [12]:
twitter_archive.rating_numerator.value_counts() 

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [13]:
#this tweet numerators need to be fixed
twitter_archive.query('rating_numerator == 75').text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)

,0,1
340,9.75,10
695,9.75,10


In [14]:
# numerators are not 75, 9.75.
twitter_archive.query('rating_numerator == 75')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None


In [15]:
# outliers
twitter_archive.query('rating_numerator == 1776')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,None,None,None,None


In [16]:
# denominators should be 10
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

quality:
- `tweet_id` datatype needs to be object (str)
- `timestamp` column is object, not datetime
- `timestamp` column contains +0000 which looks unnecessary
- retweet or reply tweets are unnecessary (only original dog ratings)
- `name` column has some strange names (a, an etc.)
- some `rating_numerator` has outliers
- `rating_numerator` of id 786709082849828864 is decimal
- some`rating_denominator` are not 10
- `doggo`, `floofer`, `pupper`, `puppo` columns (dog_stage column) have missing values, and some values are None instead of NaN

tidiness:
- `doggo`, `floofer` `pupper`, `puppo` column should be one column

## image_prediction

In [18]:
image_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [19]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


- some column headers are hard to understand(`img_num`, `p1`, `p1_conf`, `p1_dog`, etc.)
- `tweet_id` datatype needs to be object
- dog breed names in `p1`, `p2`, `p3` columns are inconsistent (lowercase, uppercase)

## tweepy_data

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tweepy_data.head(1)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 4196983835, 'id_str': '4196983835', 'na...",NaN,NaN,None,NaN,False,7703,36183,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN


In [21]:
tweepy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     2331 non-null   datetime64[ns, UTC]
 1   id                             2331 non-null   int64              
 2   id_str                         2331 non-null   int64              
 3   full_text                      2331 non-null   object             
 4   truncated                      2331 non-null   bool               
 5   display_text_range             2331 non-null   object             
 6   entities                       2331 non-null   object             
 7   extended_entities              2059 non-null   object             
 8   source                         2331 non-null   object             
 9   in_reply_to_status_id          77 non-null     float64            
 10  in_reply_to_status_id_st

## Quality Issues

**twitter_archive** 

1- `tweet_id` datatype needs to be object (str)\
2- `timestamp` column is object, not datetime\
3- `timestamp` column contains +0000 which looks unnecessary\
4- retweet or reply tweets are unnecessary (only original dog ratings)\
5- `name` column has some strange names (a, an etc.)\
6- some `rating_numerator` has too high, or too lowscore\
7- `rating_numerator` of id 786709082849828864 is decimal\
8- some`rating_denominator` are not 10\
9- `doggo`, `floofer`, `pupper`, `puppo` columns (dog_stage column) have missing values, and some values are None instead of NaN

**image_prediction**

1- some column headers are hard to understand(`img_num`, `p1`, `p1_conf`, `p1_dog`, etc.)\
2- `tweet_id` datatype needs to be object\
3- dog breed names in `p1`, `p2`, `p3` columns are inconsistent (lowercase, uppercase)

**tweepy_data**

1- drop unneccesary columns (keep `id` , `retweet_count`, `favorite_count` columns)\
2- `id` column datatype needs to be object\
3- change id column name to `tweet_id`

## Tidiness

#### twitter_archive

1- `doggo`, `floofer`, `pupper`, `puppo` columns instead of a single column dog_stage
2- `rating_numerator`, `rating_denominator` columns should be one column(rating)

**After taking care of dirty and untidy data, all tables need to be merged.
According to the rule of tidy data:
1: each variables forms a column
2: each observation forms a row
3: each type of observational unit forms a table

And `twitter_archive`, `image_prediction`, `tweepy_data` are all about WeRateDog tweet information, hence it breaks the 3rd rule. I should merge all of them. 

# Data cleaning

Data cleaning has three steps.

**Define**: convert our assessments into defined cleaning tasks. These definitions also serve as an instruction list so others (or yourself in the future) can look at your work and reproduce it.

**Code**: convert those definitions to code and run that code.

**Test**: test your dataset, visually or with code, to make sure your cleaning operations worked.

In [22]:
# make a copy

In [23]:
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweepy_data_clean = tweepy_data.copy()

## twitter_archive_clean

### 1- `tweet_id` datatype needs to be object (str)\
### 2- `timestamp` column is object, not datetime\
### 3- `timestamp` column contains +0000 which looks unnecessary

#### Define

Change datatype from integer to string
Change datatype from string to datetime
Slice data

#### Code

In [24]:
# id to string
twitter_archive_clean.tweet_id = twitter_archive_clean.tweet_id.astype(str)
# remove +0000
twitter_archive_clean.timestamp = twitter_archive_clean.timestamp.str[:-6]
# timestamp to datetime
twitter_archive_clean.timestamp = pd.to_datetime(twitter_archive_clean.timestamp)

#### Test

In [25]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   timestamp                   2356 non-null   datetime64[ns]
 4   source                      2356 non-null   object        
 5   text                        2356 non-null   object        
 6   retweeted_status_id         181 non-null    float64       
 7   retweeted_status_user_id    181 non-null    float64       
 8   retweeted_status_timestamp  181 non-null    object        
 9   expanded_urls               2297 non-null   object        
 10  rating_numerator            2356 non-null   int64         
 11  rating_denominator          2356 non-null   int64       

In [26]:
twitter_archive_clean.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


### 4- retweet or reply tweets are unnecessary (only original dog ratings)

#### Define

Drop retweets and replies, then drop unnecessary columns

#### Code

In [27]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   object        
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   timestamp                   2356 non-null   datetime64[ns]
 4   source                      2356 non-null   object        
 5   text                        2356 non-null   object        
 6   retweeted_status_id         181 non-null    float64       
 7   retweeted_status_user_id    181 non-null    float64       
 8   retweeted_status_timestamp  181 non-null    object        
 9   expanded_urls               2297 non-null   object        
 10  rating_numerator            2356 non-null   int64         
 11  rating_denominator          2356 non-null   int64       

In [28]:
# drop 78 replies
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.in_reply_to_status_id.isnull()]

In [29]:
# drop 181 retweets
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.isnull()]

In [30]:
# 2356 - 78 - 181 = 2097
len(twitter_archive_clean)

2097

In [31]:
# drop unnecessary columns
coldrop = twitter_archive_clean.iloc[:,np.r_[1,2,6:9]].columns
twitter_archive_clean.drop(coldrop, axis = 1,inplace = True)

#### Test

In [32]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2097 non-null   object        
 1   timestamp           2097 non-null   datetime64[ns]
 2   source              2097 non-null   object        
 3   text                2097 non-null   object        
 4   expanded_urls       2094 non-null   object        
 5   rating_numerator    2097 non-null   int64         
 6   rating_denominator  2097 non-null   int64         
 7   name                2097 non-null   object        
 8   doggo               2097 non-null   object        
 9   floofer             2097 non-null   object        
 10  pupper              2097 non-null   object        
 11  puppo               2097 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 213.0+ KB


### 5- name column has some strange names (a, an etc.)

#### Define

All dog names are capitalized, and some words extracted as names by mistake are all lowercase.
Make a filter to figure out all lowercase names and drop those names.

#### Code

In [33]:
# filter out all lowercase names
name_filter = twitter_archive_clean.name.str.islower()
column_name = 'name'
# drop lowercase names
twitter_archive_clean.loc[name_filter, column_name] = np.nan

#### Test

In [34]:
twitter_archive_clean.name.value_counts()

None              603
Charlie            11
Lucy               11
Cooper             10
Oliver             10
Tucker              9
Penny               9
Sadie               8
Winston             8
Lola                8
Toby                7
Daisy               7
Stanley             6
Oscar               6
Koda                6
Jax                 6
Bo                  6
Bailey              6
Bella               6
Louis               5
Chester             5
Dave                5
Rusty               5
Scout               5
Buddy               5
Leo                 5
Milo                5
Bentley             5
Clark               4
Clarence            4
Oakley              4
Gus                 4
Dexter              4
Finn                4
Brody               4
Maggie              4
Archie              4
Scooter             4
Winnie              4
Gary                4
Boomer              4
Bear                4
Reggie              4
Sophie              4
Larry               4
Jack      

### 6- some rating_numerator has too high, or too lowscore
### 7- rating_numerator of id 786709082849828864 is decimal
### 8- some rating_denominator are not 10

#### Define

extract correct rating score from `text` column by using regular expressions, replace it to existing rating columns\
determine each tweet score by computing tweet score ratio\
drop outliers if possible

#### Code

In [40]:
twitter_archive_clean.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)

,0,1
0,13,10
1,13,10
2,12,10
3,13,10
4,12,10
5,13,10
6,13,10
7,13,10
8,13,10
9,14,10


In [53]:
# get correct ratings
correct_rating = twitter_archive_clean.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)
twitter_archive_clean['rating_numerator'] = correct_rating
# some numerators are float
twitter_archive_clean['rating_numerator'] = twitter_archive_clean['rating_numerator'].astype(float)

In [57]:
# since denominators are not consistent, it is hard to determine which tweet has better score.
# So I'd like to compute numerator / denominator ratio to see tweets score.
twitter_archive_clean['rating_ratio'] = twitter_archive_clean.rating_numerator / twitter_archive_clean.rating_denominator

In [59]:
# there are some outliers, but it is much easier to see tweet score.
# I will not drop outliers since there are only 3 (0.00 , 177.60 , 42.00) , there should be no problem to do an analysis
twitter_archive_clean.rating_ratio.value_counts()


1.200000      490
1.000000      438
1.100000      418
1.300000      287
0.900000      153
0.800000       98
0.700000       51
1.400000       38
0.500000       34
0.600000       32
0.300000       19
0.400000       15
0.200000       10
0.100000        4
1.350000        1
3.428571        1
1.127000        1
0.636364        1
1.126000        1
177.600000      1
42.000000       1
0.975000        1
0.000000        1
0.818182        1
Name: rating_ratio, dtype: int64

### 9- expanded_urls has missing data and urls outside of twitter

#### Define

In [36]:
# 1 - two column for tweet id( id and id_str)
import requests
id = 'https://twitter.com/anyuser/status/892420643555336193'
id_str = 'https://twitter.com/anyuser/status/892420643555336192'
response_id = requests.get(id)
response_id_str = requests.get(id_str)

print(response_id,response_id_str)

<Response [200]> <Response [200]>


In [37]:
# Change datatype od timestamp column to datetime
pass
twitter_archive['timestamp'] = pd.to_datetime(twitter_archive['timestamp'])